In [67]:
import yfinance as yf
import pandas as pd
import zipfile
import os
import tabula


In [66]:
import requests
from bs4 import BeautifulSoup
import re

# Polish month name to number mapping
months = {
    'stycznia': '01',
    'lutego': '02',
    'marca': '03',
    'kwietnia': '04',
    'maja': '05',
    'czerwca': '06',
    'lipca': '07',
    'sierpnia': '08',
    'września': '09',
    'października': '10',
    'listopada': '11',
    'grudnia': '12'
}

# Fetch the webpage
url = "https://gpwbenchmark.pl/historyczne-portfele-indeksow"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

# Find all table cells in the index table
rows = soup.select("table#group1 tbody tr td")
revision_dates = []

for cell in rows:
    text = cell.get_text(strip=True)
    match = re.search(r'po rewizji\s*(\d{1,2}) (\w+) (\d{4})', text)
    if match:
        day, month_pl, year = match.groups()
        month = months.get(month_pl.lower())
        if month:
            formatted = f"{year}_{month}_{int(day):02d}"
            revision_dates.append(formatted)

# Print the list of dates
rd = revision_dates[:-8]
rd = [d for d in rd  if '06' in d]
rd


['2024_06_21',
 '2023_06_16',
 '2022_06_17',
 '2021_06_18',
 '2020_06_19',
 '2019_06_21',
 '2018_06_15',
 '2017_06_16',
 '2016_06_17',
 '2015_06_19',
 '2014_06_20',
 '2013_06_21',
 '2012_06_15']

In [22]:
base_dir = "data"

def fetch_tickers_from_wikipedia(url, table_index, column_name):
    """Scrape tickers from a Wikipedia table."""
    try:
        tables = pd.read_html(url)
        return tables[table_index][column_name].dropna().tolist()
    except Exception as e:
        print(f"Error fetching tickers from {url}: {e}")
        return []

indices = {
    #"ftse100": 
    
    # {"url": "https://en.wikipedia.org/wiki/FTSE_100_Index",
    #     "table_index": 4,
    #     "column": "Ticker",
    #     "suffix": ".L"
    # },
    # "nikkei225": {
    #     "url": "https://en.wikipedia.org/wiki/Nikkei_225",
    #     "table_index": 1,
    #     "column": "Ticker",
    #     "suffix": ".T"
    # }
    # "sp500": {
    #     "url": "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies",
    #     "table_index": 0,
    #     "column": "Symbol",
    #     "suffix": ""
    # },

    'sp500_old': {
        "url": "https://en.wikipedia.org/w/index.php?title=List_of_S%26P_500_companies&oldid=871568811",
        "table_index": 0,
        "column": "Symbol",
        "suffix": ""
    },    
}

for index, meta in indices.items():
    print(f"\n Downloading data for: {index.upper()}")
    tickers = fetch_tickers_from_wikipedia(meta["url"], meta["table_index"], meta["column"])
    # tickers = pd.read_csv(r'C:\\physics_masters\\nikkei_stock_average_weight_en.csv', encoding='utf-8')['Code'].tolist()
    tickers = [str(t).strip() for t in tickers]
    print(tickers)
    index_dir = os.path.join(base_dir, index)
    os.makedirs(index_dir, exist_ok=True)

    for ticker in tickers:
        full_ticker = ticker + meta["suffix"]
        try:
            df = yf.download(full_ticker, start="2000-01-01", end="2010-12-31", interval="1d")
            if not df.empty:
                df.to_csv(os.path.join(index_dir, f"{ticker}.csv"))
                print(f"Saved: {ticker}")
        except Exception as e:
            print(f"❌ {ticker}: {e}")


['MMM', 'ABT', 'ABBV', 'ABMD', 'ACN', 'ATVI', 'ADBE', 'AMD', 'AAP', 'AES', 'AMG', 'AFL', 'A', 'APD', 'AKAM', 'ALK', 'ALB', 'ARE', 'ALXN', 'ALGN', 'ALLE', 'AGN', 'ADS', 'LNT', 'ALL', 'GOOGL', 'GOOG', 'MO', 'AMZN', 'AEE', 'AAL', 'AEP', 'AXP', 'AIG', 'AMT', 'AWK', 'AMP', 'ABC', 'AME', 'AMGN', 'APH', 'APC', 'ADI', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'AIV', 'AAPL', 'AMAT', 'APTV', 'ADM', 'ARNC', 'ANET', 'AJG', 'AIZ', 'T', 'ADSK', 'ADP', 'AZO', 'AVB', 'AVY', 'BHGE', 'BLL', 'BAC', 'BK', 'BAX', 'BBT', 'BDX', 'BRK-B', 'BBY', 'BIIB', 'BLK', 'HRB', 'BA', 'BKNG', 'BWA', 'BXP', 'BSX', 'BHF', 'BMY', 'AVGO', 'BR', 'BF-B', 'CHRW', 'COG', 'CDNS', 'CPB', 'COF', 'CAH', 'KMX', 'CCL', 'CAT', 'CBOE', 'CBRE', 'CBS', 'CELG', 'CNC', 'CNP', 'CTL', 'CERN', 'CF', 'SCHW', 'CHTR', 'CVX', 'CMG', 'CB', 'CHD', 'CI', 'XEC', 'CINF', 'CTAS', 'CSCO', 'C', 'CFG', 'CTXS', 'CLX', 'CME', 'CMS', 'KO', 'CTSH', 'CL', 'CMCSA', 'CMA', 'CAG', 'CXO', 'COP', 'ED', 'STZ', 'COO', 'CPRT', 'GLW', 'COST', 'COTY', 'CCI', 'CSX', 'CMI', 'C

[*********************100%***********************]  1 of 1 completed


Saved: MMM


[*********************100%***********************]  1 of 1 completed


Saved: ABT


...

[*********************100%***********************]  1 of 1 completed


Saved: ZBH


[*********************100%***********************]  1 of 1 completed


Saved: ZION


[*********************100%***********************]  1 of 1 completed

1 Failed download:
['ZTS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2000-01-01 -> 2010-12-31) (Yahoo error = "Data doesn\'t exist for startDate = 946702800, endDate = 1293771600")')
